In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
files.upload()

Saving Mall_Customers.csv to Mall_Customers (1).csv


{'Mall_Customers (1).csv': b'CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)\r\n1,Male,19,15,39\r\n2,Male,21,15,81\r\n3,Female,20,16,6\r\n4,Female,23,16,77\r\n5,Female,31,17,40\r\n6,Female,22,17,76\r\n7,Female,35,18,6\r\n8,Female,23,18,94\r\n9,Male,64,19,3\r\n10,Female,30,19,72\r\n11,Male,67,19,14\r\n12,Female,35,19,99\r\n13,Female,58,20,15\r\n14,Female,24,20,77\r\n15,Male,37,20,13\r\n16,Male,22,20,79\r\n17,Female,35,21,35\r\n18,Male,20,21,66\r\n19,Male,52,23,29\r\n20,Female,35,23,98\r\n21,Male,35,24,35\r\n22,Male,25,24,73\r\n23,Female,46,25,5\r\n24,Male,31,25,73\r\n25,Female,54,28,14\r\n26,Male,29,28,82\r\n27,Female,45,28,32\r\n28,Male,35,28,61\r\n29,Female,40,29,31\r\n30,Female,23,29,87\r\n31,Male,60,30,4\r\n32,Female,21,30,73\r\n33,Male,53,33,4\r\n34,Male,18,33,92\r\n35,Female,49,33,14\r\n36,Female,21,33,81\r\n37,Female,42,34,17\r\n38,Female,30,34,73\r\n39,Female,36,37,26\r\n40,Female,20,37,75\r\n41,Female,65,38,35\r\n42,Male,24,38,92\r\n43,Male,48,39,36\r\n44,Female

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

#Creamos una Spark Instance

spark = SparkSession.builder.appName('Classification').getOrCreate()

In [ ]:
df= spark.read.csv('Mall_Customers.csv',header=True,inferSchema=True)
df

DataFrame[CustomerID: int, Gender: string, Age: int, Annual Income (k$): int, Spending Score (1-100): int]

In [ ]:
df.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Annual Income (k$): integer (nullable = true)
 |-- Spending Score (1-100): integer (nullable = true)



In [ ]:
df.show()

+----------+------+---+------------------+----------------------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
|         6|Female| 22|                17|                    76|
|         7|Female| 35|                18|                     6|
|         8|Female| 23|                18|                    94|
|         9|  Male| 64|                19|                     3|
|        10|Female| 30|                19|                    72|
|        11|  Male| 67|                19|                    14|
|        12|Female| 35|                19|                    99|
|        1

In [ ]:
df.groupby('Gender').count().show()

+------+-----+
|Gender|count|
+------+-----+
|Female|  112|
|  Male|   88|
+------+-----+



In [ ]:
for i in df.columns:
  print('Numero de filas nulas en '+str(i)+' is: '+str(df.filter(df[i].isNull()).count()))

Numero de filas nulas en CustomerID is: 0
Numero de filas nulas en Gender is: 0
Numero de filas nulas en Age is: 0
Numero de filas nulas en Annual Income (k$) is: 0
Numero de filas nulas en Spending Score (1-100) is: 0


In [ ]:
#Filas del dataframe
df.count()

200

In [ ]:
#Convertimos la variables categorica en numericas

from pyspark.ml.feature import StringIndexer

stringIndex= StringIndexer(inputCols=['Gender'],outputCols=['Gender_new'])

stringIndex_model= stringIndex.fit(df)

df_new= stringIndex_model.transform(df)
df_new.show()

+----------+------+---+------------------+----------------------+----------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|Gender_new|
+----------+------+---+------------------+----------------------+----------+
|         1|  Male| 19|                15|                    39|       1.0|
|         2|  Male| 21|                15|                    81|       1.0|
|         3|Female| 20|                16|                     6|       0.0|
|         4|Female| 23|                16|                    77|       0.0|
|         5|Female| 31|                17|                    40|       0.0|
|         6|Female| 22|                17|                    76|       0.0|
|         7|Female| 35|                18|                     6|       0.0|
|         8|Female| 23|                18|                    94|       0.0|
|         9|  Male| 64|                19|                     3|       1.0|
|        10|Female| 30|                19|                    72|       0.0|

In [ ]:
#Borramos las columnas 'Customer ID' y 'Gender'

df_new_2=df_new.drop('Gender','CustomerID')

df_new_2.show()

+---+------------------+----------------------+----------+
|Age|Annual Income (k$)|Spending Score (1-100)|Gender_new|
+---+------------------+----------------------+----------+
| 19|                15|                    39|       1.0|
| 21|                15|                    81|       1.0|
| 20|                16|                     6|       0.0|
| 23|                16|                    77|       0.0|
| 31|                17|                    40|       0.0|
| 22|                17|                    76|       0.0|
| 35|                18|                     6|       0.0|
| 23|                18|                    94|       0.0|
| 64|                19|                     3|       1.0|
| 30|                19|                    72|       0.0|
| 67|                19|                    14|       1.0|
| 35|                19|                    99|       0.0|
| 58|                20|                    15|       0.0|
| 24|                20|                    77|       0.

In [ ]:
#Cambiamos el nombre de Gender_new a Gender

df_new_3=df_new_2.withColumnRenamed('Gender_new','Gender')
df_new_3.show()

+---+------------------+----------------------+------+
|Age|Annual Income (k$)|Spending Score (1-100)|Gender|
+---+------------------+----------------------+------+
| 19|                15|                    39|   1.0|
| 21|                15|                    81|   1.0|
| 20|                16|                     6|   0.0|
| 23|                16|                    77|   0.0|
| 31|                17|                    40|   0.0|
| 22|                17|                    76|   0.0|
| 35|                18|                     6|   0.0|
| 23|                18|                    94|   0.0|
| 64|                19|                     3|   1.0|
| 30|                19|                    72|   0.0|
| 67|                19|                    14|   1.0|
| 35|                19|                    99|   0.0|
| 58|                20|                    15|   0.0|
| 24|                20|                    77|   0.0|
| 37|                20|                    13|   1.0|
| 22|     

In [ ]:
df_new_3.groupby('Gender').count().show()

+------+-----+
|Gender|count|
+------+-----+
|   0.0|  112|
|   1.0|   88|
+------+-----+



In [ ]:
# se crea una columna que ensambla Age, Annual Income  y Spending Score con VectorAssembler

from pyspark.ml.feature import VectorAssembler

VA = VectorAssembler(inputCols=df_new_3.columns[:-1], outputCol='features') #Ensambla todas las variables menos Gender

df_new_3= VA.transform(df_new_3)

df_new_3.show(5)

+---+------------------+----------------------+------+----------------+
|Age|Annual Income (k$)|Spending Score (1-100)|Gender|        features|
+---+------------------+----------------------+------+----------------+
| 19|                15|                    39|   1.0|[19.0,15.0,39.0]|
| 21|                15|                    81|   1.0|[21.0,15.0,81.0]|
| 20|                16|                     6|   0.0| [20.0,16.0,6.0]|
| 23|                16|                    77|   0.0|[23.0,16.0,77.0]|
| 31|                17|                    40|   0.0|[31.0,17.0,40.0]|
+---+------------------+----------------------+------+----------------+
only showing top 5 rows



In [ ]:
#Se deja solo la columna features y Gender

df_new_3=df_new_3.select('features', 'Gender')

df_new_3.show()

+----------------+------+
|        features|Gender|
+----------------+------+
|[19.0,15.0,39.0]|   1.0|
|[21.0,15.0,81.0]|   1.0|
| [20.0,16.0,6.0]|   0.0|
|[23.0,16.0,77.0]|   0.0|
|[31.0,17.0,40.0]|   0.0|
|[22.0,17.0,76.0]|   0.0|
| [35.0,18.0,6.0]|   0.0|
|[23.0,18.0,94.0]|   0.0|
| [64.0,19.0,3.0]|   1.0|
|[30.0,19.0,72.0]|   0.0|
|[67.0,19.0,14.0]|   1.0|
|[35.0,19.0,99.0]|   0.0|
|[58.0,20.0,15.0]|   0.0|
|[24.0,20.0,77.0]|   0.0|
|[37.0,20.0,13.0]|   1.0|
|[22.0,20.0,79.0]|   1.0|
|[35.0,21.0,35.0]|   0.0|
|[20.0,21.0,66.0]|   1.0|
|[52.0,23.0,29.0]|   1.0|
|[35.0,23.0,98.0]|   0.0|
+----------------+------+
only showing top 20 rows



In [ ]:
# Dividimos el dataset en train y test

train_df,test_df= df_new_3.randomSplit([0.8,0.2])
print(train_df.count())
print(test_df.count())

154
46


In [ ]:
#Se evalua la performance del modelo

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

perf = MulticlassClassificationEvaluator(labelCol='Gender', metricName='accuracy')

# **MODELO DE REGRESION LOGISTICA**

RIDGE

In [ ]:
# testing of the Logistic Regression model with elastic net regularisation:
# - elastic net parameter set to 1 is equivalent to Lasso
# - elastic net parameter set to 0 is equivalent to Ridge

from pyspark.ml.classification import LogisticRegression

#RIDGE

ridge_reg = LogisticRegression(labelCol='Gender',
                        maxIter=100,
                        elasticNetParam=0,  # parametro 0 es Ridge
                        regParam=0.03)

model = ridge_reg.fit(train_df)
pred = model.transform(test_df)
perf.evaluate(pred)

0.4782608695652174

In [ ]:
#LASSO

lasso_reg = LogisticRegression(labelCol='Gender',
                           maxIter=100,
                           elasticNetParam=1, # parametro 1 es Lasso
                           regParam=0.0003)

model = lasso_reg.fit(train_df)
pred = model.transform(test_df)
perf.evaluate(pred)

0.43478260869565216